In [1]:
import egttools as egt
import numpy as np
import math
import matplotlib.pyplot as plt
import scipy as sp

import run
from run import CRDWithExecutor, Cooperator, Defector, Executor

Z = 100         # Population size
N  = 4           # Group size
b  = 1.           # Endowment (individual's money/funds/...)
c  = 0.1         # Amount of money individuals contribute
Mc = 0.3         # Minimum collective contribution
M  = 3.           # OR Minimum number of cooperators
r  = 0.2         # If minimum is not met: All group participants lose their endowment with probability r, else: individuals retain their endowments
pi_t = 0.03
pi_e = 0.3
n_e = 0.25
alpha = 1.
mu    = 1/Z
beta = 5.

game = CRDWithExecutor(
    strategies=[Defector(c, b), Executor(c, b, pi_t, pi_e, alpha), Cooperator(c, b)],
    initial_endowment=b,
    population_size=Z,
    group_size=N,
    cost=c,
    risk=r,
    alpha=alpha,
    cooperation_threshold=M,
    enhancement_factor=1,
    pi_t=pi_t,
    pi_e=pi_e,
    n_e=n_e,
    mu=mu)

In [84]:
# Make population and group
current_population = np.array([33, 34, 33], dtype=int)
pop_id = egt.calculate_state(100, current_population)

dist = sp.stats.multivariate_hypergeom(current_population, game.N)
group = dist.rvs()[0]
group_id = egt.calculate_state(game.N, group,)

print(group)

[2 1 1]


In [85]:
# Test fitness difference
game.set_population_state(pop_id)
payoffs = game.calculate_payoffs()
evolver = egt.analytical.StochDynamics(
        3, 
        np.array(payoffs), 
        pop_size=game.Z, 
        group_size=game.N, 
        mu=game.mu)

evolver.full_fitness_difference_group(0, 2, population_state=current_population)

-0.20397834860279696

In [168]:
# Generating a random pair of 2 strategies from population
sample = sp.stats.multivariate_hypergeom.rvs(m=current_population, n=2)
print(sample)
idvs = [0 for _ in range(sample[0])]+[1 for _ in range(sample[1])]+[2 for _ in range(sample[2])]
np.random.shuffle(idvs)
print(idvs)

[1 1 0]
[1, 0]


In [176]:
# Calculating group achievement from stationary distribution estimate
import pickle
with open("alpha=0.pickle", 'rb') as f:
        sd = pickle.load(f)

group_achievement = sum([
        sd[i]*game.aG(i) for i in range(len(sd))
    ])
print("Alpha = 0:",group_achievement)

with open("alpha=1.pickle", 'rb') as f:
        sd = pickle.load(f)

group_achievement = sum([
        sd[i]*game.aG(i) for i in range(len(sd))
    ])
print("Alpha = 1:",group_achievement)

Alpha = 0: 0.15532937031440683
Alpha = 1: 0.04455053032476516
